# Role de footprints dans la création des objets

Reprenons l'exemple précédent de création d'une *ressource* :

```python
pe_resource = fp.proxy.resource(kind='gridpoint',
                                term=3,
                                geometry='euroc25',
                                nativefmt='grib',
                                model='arpege',
                                cutoff='production',
                                date='2017060118',
                                origin='historic')
```

On appelle une méthode générique qui crée le "bon" objet en fonction des attributs qui lui sont passés : cela fait fortement penser au patron de conception de la Fabrique (*factory design pattern* pour les anglophones). C'est une façon de décrire le rôle du package **footprints**, néanmoins **footprints** fait bien plus que d'implémenter une Fabrique (notamment en terme de flexibilité).

## Description des objets gérés par footprints

Nous allons maintenant commencer notre démonstration du package footprints, la moindre des choses est de charger le package :

In [1]:
import footprints as fp

En soit, le fait de charger le package **footprints** ne fait rien. Tout se joue lors de l'écriture des classes concernées et de la création des objets

### Classes, Collecteurs et Objets

Voici un exemple de classe adossée au package footprints :

In [2]:
class Observations(fp.FootprintBase):
    
    _collector = ('fakeresource', )  # collecteur
    _footprint = dict(  # empreinte
        info = 'Any kind of insitu observations',
        attr = dict(
            kind = dict(
                values = ['observations', ],
                info   = 'The kind of resource',
            )
        )
    )

* Toute classe "footprintisée" doit hériter de **FootprintBase**
* Une classe "footprintisée" est ratachée à un (ou plusieurs) *collecteur*. Chaque collecteur est un objet de classe **footprints.collectors.Collector** qui maintient, pour un thème donné (ici "fakeresource") un liste des différentes classes qui ont été déclarées.
* Chaque classe "footprintisée" doit être munie d'une empreinte qui spécifie quels sont les attributs d'une classe, ...

La récupération d'un objet collecteur peut se faire via le module **collectors** :

In [3]:
print fp.collectors.get(tag='fakeresource')

En générale, une autre syntaxe est préfére :

In [4]:
from footprints import proxy as fpx
print fpx.fakeresources  # Attention au "s" à la fin

Quelles sont les classes ratachées au collecteur "fakeresource" ?

In [5]:
print fpx.fakeresources.items()

[<class '__main__.Observations'>]


On retrouve bien la classe que nous avons définie précédement...

En tant que catalogue des différentes classes disponibles, c'est le *collecteur* qui permet la création d'objets :

In [6]:
obs_obj1 = fpx.fakeresources.load(kind='observations')
print obs_obj1

<__main__.Observations object at 0x7fc0a1f04dd0 | footprint=1>


Là aussi le package **footprints** fournie une syntaxe simplfiée qui est souvent préférée à la précédente :

In [7]:
obs_obj2 = fpx.fakeresource(kind='observations')  # Attention : pas de "s" à la fin
print obs_obj2

<__main__.Observations object at 0x7fc0a1f04e10 | footprint=1>


A partir de ce simple exemple on peut entrevoir l'éventualité d'erreurs... Que se passe t'il si l'on demande un objet dont le *kind* n'existe pas...

In [8]:
failer1 = fpx.fakeresource(kind='toto')
print 'failer1 is:', str(failer1)

# [2017/08/21-14:08:30][footprints.collectors][pickup:0286][WARNING]: No fakeresource found in description 
      dict(
          fakeresource = None, 
          kind = 'toto',
      )



 Report Footprint-Fakeresource: 

     __main__.Observations
         kind       : {'args': 'toto', 'why': 'Not in values'}

failer1 is: None


En cas d'erreur, le *collecteur* renvoi *None* et un message d'erreur est affiché détaillant pourquoi il n'est pas possible d'instancier telle ou telle classe.

### Héritage et footprints prédéfinies

Dans un langage comme Python, l'utilisation de l'héritage de classe est plus que recomandée. Il faut donc que le package **footprints** gère ce cas de façon optimale. En pratique, toute classe héritant d'une classe "footprintisée" hérite aussi de l'empreinte de la classe mère. Cette empreinte peut ensuite être altérée ou étendue au niveau de la classe fille.

**footprints** offre aussi la possibilité de créer classes abstraites. Cela permet de définir des classes indispensables à la structure de l'arbre d'héritage tout en ayant la certitude que celles-ci ne seront jamais instanciées.

Certains attributs sont des "classiques", on peut imaginer de les prédéfinir.

Dans un contexte de prévision numérique, on peut par exemple considérer que le *cutoff* est un attribut classique (assimilation ou production) et vouloir le prédéfinir :

In [9]:
a_cutoff = dict(
    info     = "The cutoff type of the generating process.",
    values   = ['assim', 'production', ],
)
cutoff = fp.Footprint(info = 'Abstract cutoff', attr = dict(cutoff = a_cutoff))

Cette attribut prédéfini peut dès lors être utilisé pour définir une classe abstraite servant de base à différents champs modèle :

In [10]:
class ModelState(fp.FootprintBase):

    _collector = ('fakeresource', )
    _abstract = True
    _footprint = [
        cutoff,
        dict(
            info = 'Abstract Model State',
            attr = dict(
                kind = dict(
                    info = 'The kind of modelstates we are dealing with',
                ),
                date = dict(
                    info = 'The base date of this model state'
                ),
            )
        )
    ]


Héritons de cette classe pour créer des conditions initiales :

In [11]:
class InitialConditions(ModelState):
    
    _footprint = dict(
        attr = dict(
            kind = dict(
                values = ['initial_conditions', ]
            ),       
        )
    )    

In [12]:
ic_obj = fpx.fakeresource(kind='initial_conditions', date='2017070100', cutoff='assim')
print ic_obj
print 'kind={0.kind:s}, date={0.date:s}, cutoff={0.cutoff:s}'.format(ic_obj)

<__main__.InitialConditions object at 0x7fc09aa1a650 | footprint=3>
kind=initial_conditions, date=2017070100, cutoff=assim


### Description détaillée des attributs

Il est possible d'utiliser un grand nombre de clés lors de la description des attributs :

  * **type** : type (au sens de Python) de l'attribut (*str* par défaut)
  * **outcast** : opposé de *values* : on précise les valeurs interdites et non celles autorisées
  * **optional** : un attribut peut être optional (par défaut un attribut est obligatoire)
  * **default** : si ``optional=True`` on peut préciser une valeur par défaut (*None* par défaut)
  * **alias** : lors de la phase de création des objets (résolution des empreintes), un même attribut peut avoir plusieurs alias
  * **remap** : il est possible de faire pointer une valeur autorisée vers une autre



In [13]:
class Historic(ModelState):
    
    _footprint = dict(
        attr = dict(
            kind = dict(
                values = ['historic', 'forecastfile', ],
                remap = dict(forecastfile='historic', ),
            ),
            term = dict(
                info = "The forecast's term",
                type = int,
                alias = ('fcterm', 'forecastterm', ),
            ),
            nativefmt = dict(
                info = "The storage's format.",
                values = ['grib', 'fa', 'netcdf', ],
                optional = True,
                default = 'grib',
            ),
        )
    )

#### Demonstration de *remap* et *optional*

Rappel:

  * pour *kind*, ``remap = dict(forecastfile='historic', )`` ;
  * pour *nativefmt*, ``optional = True`` et ``default = 'grib'``.

In [14]:
obj_hst1 = fpx.fakeresource(kind='forecastfile',
                            cutoff='assim', date='20170701',
                            term=6)
print obj_hst1, "object's kind:     ", obj_hst1.kind
print obj_hst1, "object's nativefmt:", obj_hst1.nativefmt

<__main__.Historic object at 0x7fc09aa1a410 | footprint=5> object's kind:      historic
<__main__.Historic object at 0x7fc09aa1a410 | footprint=5> object's nativefmt: grib


#### Demonstration de *alias*

Rappel: pour *term*, ``alias = ('fcterm', 'forecastterm', ),``.

In [15]:
obj_hst1 = fpx.fakeresource(kind='forecastfile',
                            cutoff='assim', date='20170701',
                            fcterm=6)
print obj_hst1, "object's term:  ", obj_hst1.term
try:
    print obj_hst1, "object's fcterm:", obj_hst1.fcterm
except AttributeError:
    print "Notice that 'fcterm' does not really exists."

<__main__.Historic object at 0x7fc09aa21790 | footprint=5> object's term:   6
<__main__.Historic object at 0x7fc09aa21790 | footprint=5> object's fcterm: Notice that 'fcterm' does not really exists.


#### Demonstration de *type*

Rappel: pour *term*, ``type = int,``

In [16]:
# Ok
obj_hst1 = fpx.fakeresource(kind='forecastfile', cutoff='assim', date='20170701',
                            term='6')
print "{0!r} object's term is {0.term:d}\n".format(obj_hst1)
# KO
print 'This will fail:'
obj_hst1 = fpx.fakeresource(kind='forecastfile', cutoff='assim', date='20170701',
                            term='This is not an integer !')

# [2017/08/21-14:08:31][footprints.collectors][pickup:0286][WARNING]: No fakeresource found in description 
      dict(
          cutoff = 'assim', 
          date = '20170701', 
          fakeresource = None, 
          kind = 'forecastfile', 
          term = 'This is not an integer !',
      )


<__main__.Historic object at 0x7fc09aa21550> object's term is 6

This will fail:

 Report Footprint-Fakeresource: 

     __main__.Historic
         term       : {'args': ('int', 'This is not an integer !'), 'why': 'Could not reclass'}

     __main__.InitialConditions
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}

     __main__.Observations
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}



Ici nous avons utilisé un type simple qui préexistait dans le langage Python (*int*), il est bien sûr possible d'utiliser des types personalisés (la seule contrainte est que les types utilisés doivent être "hashable").

C'est d'ailleurs ce que nous aurions dû faire pour les attributs *date* et *term*. En effet :

  * une date n'est pas une simple chaîne de caractère (il faut vérifier que la date fournie est valide),
  * une échéance n'est malheureusement pas un entier mais peut être exprimée en heures et minutes.
  
Si il y a lieu, l'usage de types personalisés est donc fortement recommandé (voir par exemple les classes **vortex.tools.date.Date** et **vortex.tools.date.Time**)

## Notion de valeurs prédéfinies

Il est souvent utile de préciser en début de script des valeurs pré-définies pour un certain nombre d'attributs. Dans nos exemple précédents, on imagine assez bien que l'utilisateur puisse vouloir fixer une *date* et un *cutoff*.

Il suffit de définir ces valeurs via l'objet **setup** du package **footprints**.

NB: Il ne s'agit que de valeurs prédéfinis : si l'utilisateur précise une autre valeur lors de la création de l'objet, celle-ci sera prioritaire.

Définissons un *cutoff* et une *date* par défaut :

In [17]:
fp.setup.defaults =  dict(cutoff='assim', date='20170701')

# Utilisation du defaut:
obj_hst2a = fpx.fakeresource(kind='forecastfile', term=6)
print '{0!r}: cutoff={0.cutoff:s}, date={0.date:s}'.format(obj_hst2a)
# Defaut redefini manuellement pour le cutoff:
obj_hst2b = fpx.fakeresource(kind='forecastfile', cutoff='production', term=6)
print '{0!r}: cutoff={0.cutoff:s}, date={0.date:s}'.format(obj_hst2b)

<__main__.Historic object at 0x7fc09aa1a610>: cutoff=assim, date=20170701
<__main__.Historic object at 0x7fc09aa1a150>: cutoff=production, date=20170701


## Conclusion

Lors de la création de classes utilisant **footprints** :

* Bien recourir à l'héritage et déclarer comme telle les classes abstraites ;
* Pour les attributs les plus courants, définir des empreintes pré-définies et les utiliser ;
* Penser à préciser des clés "info" qui sont facultatives mais apparaissent dans la documentation automatique  ;
* Décrire les types de données le plus précisement possible ;